In [1]:
import tensorflow as tf
from tensorflow import keras
from matplotlib import pyplot as plt
import datetime
import LoadPatches
%matplotlib inline
%load_ext tensorboard

/Applications/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


**Settings**

In [2]:
image_folder = 'cellsuspensionED8'
roi_file = 'CS8_ROIs.csv'
patch_size = 23

# Clipping parameter for histogram equalisation
clip_limit = 0.03

In [3]:
originals, labels, count = LoadPatches.process_rois('CS8_ROIs.csv', 'cellsuspensionED8/', 0, 
                                                    patch_size, clip_limit)
print('count = ', count)

/Applications/anaconda3/lib/python3.6/site-packages/skimage/util/dtype.py:141: UserWarning: Possible precision loss when converting from float64 to uint16
  .format(dtypeobj_in, dtypeobj_out))


count =  883402


In [59]:
import os
import glob
import cv2
import pandas as pd
from skimage import exposure
from sklearn.feature_extraction import image

image_path = '../originals/'
seg_path = '../Anna\'s results'

filename = 'CS8_ROIs.csv'
folder = 'cellsuspensionED8/'

images = glob.glob(image_path + folder + "*.tif")
# images = []
# for name in image_list:
#     images.append(cv2.imread(name))

In [37]:
images

['../originals/cellsuspensionED8/egg3.tif',
 '../originals/cellsuspensionED8/egg2.tif',
 '../originals/cellsuspensionED8/egg6.tif',
 '../originals/cellsuspensionED8/egg5.tif']

In [54]:
csv_path = os.path.join(seg_path, filename)
regions = pd.read_csv(csv_path, index_col=0)
regions['patch_per_roi'] = (regions.height - patch_size + 1) * (regions.width - patch_size + 1)
regions['bottom_px'] = regions.ycoord + regions.height
regions['right_px'] = regions.xcoord + regions.width

regions.head(10)

,label,corner,sample,branch_length,area,xcoord,ycoord,width,height,bld,day,cond,patch_per_roi,bottom_px,right_px
1,egg2,0,0,1987.650,32752,228,28,178,184,0.060688,Day 8,Cells,25272,212,406
2,egg2,0,1,1293.367,32752,274,157,178,184,0.039490,Day 8,Cells,25272,341,452
3,egg2,0,2,1899.560,32752,943,300,178,184,0.057998,Day 8,Cells,25272,484,1121
4,egg2,0,3,1234.393,32752,221,594,178,184,0.037689,Day 8,Cells,25272,778,399
5,egg2,0,4,1753.803,32752,915,738,178,184,0.053548,Day 8,Cells,25272,922,1093
6,egg2,0,5,1531.487,32752,25,449,178,184,0.046760,Day 8,Cells,25272,633,203
7,egg2,0,6,1755.484,32752,933,196,178,184,0.053599,Day 8,Cells,25272,380,1111
8,egg2,0,7,2372.881,32752,902,700,178,184,0.072450,Day 8,Cells,25272,884,1080
9,egg2,0,8,959.646,32752,28,600,178,184,0.029300,Day 8,Cells,25272,784,206
10,egg2,0,9,1416.231,32752,414,824,178,184,0.043241,Day 8,Cells,25272,1008,592


In [51]:
regions.label.unique()

array(['egg2', 'egg3', 'egg5', 'egg6'], dtype=object)

In [73]:
# for label in regions.label.unique():
label = 'egg2'

img = cv2.imread(image_path + folder + label + ".tif")
img = exposure.equalize_adapthist(img[:, :, 1], clip_limit=clip_limit) # keep only the green channel! 

In [84]:


for _, row in regions[regions.label == label].iterrows():
    
    y = image.extract_patches_2d(img[row.ycoord : row.bottom_px, row.xcoord : row.right_px], 
                                 (patch_size, patch_size))



(25272, 23, 23)